TODO add microexons

    /projects/ps-yeolab/software/vast-tools/VASTDB/Hsa/TEMPLATES/Hsa.MIC.Template.txt

In [2]:
import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

%load_ext autoreload
%autoreload 2

# from outrigger.junctions_to_events import make_junction_exon_triples

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/home/obotvinnik/projects/singlecell_pnms/analysis/csvs_for_paper'

sj_metadata = pd.read_csv('{}/sj_metadata.csv'.format(folder), index_col=0)
sj_metadata.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,chrom,intron_start,intron_stop,strand,intron_motif,annotated
intron_location,,,,,,
chr1:14830-14969:-,chr1,14830,14969,-,GT/AG,True
chr1:15039-15795:-,chr1,15039,15795,-,GT/AG,True
chr1:228777-237053:-,chr1,228777,237053,-,GT/AG,True
chr1:320654-324287:+,chr1,320654,324287,+,GT/AG,True
chr1:320939-321031:+,chr1,320939,321031,+,GT/AG,True


In [1]:
import gffutils

In [3]:
gffutils.create_db?

In [2]:
gffutils.FeatureDB('asdf.db')

ValueError: Database file asdf.db does not exist

`intron_start` and `intron_stop` are defined as absolute coordinates (1-based, like GFF files) in the genome, regardless of strand. Thus, `intron_start` is **always** smaller than `intron_stop`. We can test the converse here, which asks if `intron_start` is *ever* larger than `intron_stop`. The result of this is 0, as it should be.

In [3]:
(sj_metadata['intron_start'] > sj_metadata['intron_stop']).sum()

0

To get exons adjacent to the introns, we need to have a column that maps to an exon coordinate. Subtracting one (1) from `intron_start` gets us the end of the exon (`exon_stop`) and adding one (1) to `intron_stop` gets us the beginning of the exon (`exon_start`)

In [4]:
sj_metadata['exon_stop'] = sj_metadata['intron_start'] - 1
sj_metadata['exon_start'] = sj_metadata['intron_stop'] + 1
sj_metadata.head()

,chrom,intron_start,intron_stop,strand,intron_motif,annotated,exon_stop,exon_start
intron_location,,,,,,,,
chr1:14830-14969:-,chr1,14830,14969,-,GT/AG,True,14829,14970
chr1:15039-15795:-,chr1,15039,15795,-,GT/AG,True,15038,15796
chr1:228777-237053:-,chr1,228777,237053,-,GT/AG,True,228776,237054
chr1:320654-324287:+,chr1,320654,324287,+,GT/AG,True,320653,324288
chr1:320939-321031:+,chr1,320939,321031,+,GT/AG,True,320938,321032


In [5]:
from joblib import Parallel, delayed
from outrigger.junctions_to_events import adjacent_junctions

Paralellization doesn't seem to be faster

In [ ]:
# %%time
# dfs = Parallel(n_jobs=4)(delayed(adjacent_junctions)(
#         x, sj_metadata, chrom='chrom')for x in v19db.features_of_type('exon')) 

In [ ]:
juction_exon_triples = pd.concat(dfs, ignore_index=True)

In [ ]:
dfs = []
for exon in v19db.features_of_type('exon'):
    dfs.append(adjacent_junctions(exon, sj_metadata))

In [12]:
junction_exon_triples = pd.concat(dfs, ignore_index=True)

In [13]:
junction_exon_triples.head()

,exon,direction,junction
0,exon:chr1:29321-29370:-,upstream,chr1:18367-29320:-
1,exon:chr1:29321-29370:-,upstream,chr1:24892-29320:-
2,exon:chr1:29321-29370:-,upstream,chr1:27186-29320:-
3,exon:chr1:29321-29370:-,upstream,chr1:18380-29320:-
4,exon:chr1:29321-29370:-,upstream,chr1:18370-29320:-


In [14]:
junction_exon_triples.to_csv('{}/junction_exon_triples.csv'.format(folder), index=False)